# Bob's LLM Remote Example

This notebook demonstrates how Bob can set up his data, grant permissions to Alice, and use the LLM remote functionality.

In [1]:
from __future__ import annotations

import os
import time
from pathlib import Path
from loguru import logger

# Import the client module
from llama_remote import client

# Helper function to create sample files
def create_sample_file(file_path, content):
    """Create a sample file with the given content."""
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    with open(file_path, 'w') as f:
        f.write(content)
    print(f"Created sample file: {file_path}")

In [2]:
# Creating Bob's client
bob_client = client("~/.syft_bob_config.json")
print(f"Connected as: {bob_client.client.email}")

Connected as: bob@openmined.org


## Create Bob's Private Business Data

In [3]:
# Bob's private data - business metrics
bob_data_dir = str(bob_client.box.client.datasite_path) + "/company_data"
bob_files = {
    f"{bob_data_dir}/financial_metrics.csv": """
quarter,revenue,expenses,profit,growth_rate
Q1-2024,1250000,875000,375000,0.05
Q2-2024,1375000,950000,425000,0.08
Q3-2024,1450000,925000,525000,0.12
Q4-2024,1650000,1050000,600000,0.15
Q1-2025,1825000,1150000,675000,0.11
""",
    f"{bob_data_dir}/strategic_plan.txt": """
CONFIDENTIAL: STRATEGIC GROWTH PLAN 2025-2027

Vision: Achieve 30% market share by end of 2027
- Q2 2025: Launch new product line targeting SMB sector
- Q4 2025: Expand operations to Asian markets
- Q2 2026: Acquire competitor XYZ Technologies 
- Q4 2026: Roll out enterprise solution with advanced AI capabilities
- Q2 2027: IPO preparation and valuation assessment

Budget allocation:
- R&D: 25%
- Marketing: 30%
- Operations: 35%
- Acquisitions: 10%

Expected outcomes:
- Revenue growth: 25% YoY
- New enterprise clients: 15-20 per quarter
- Net profit margin: Increase from 18% to 24%
"""
}

# Create the sample files
for file_path, content in bob_files.items():
    create_sample_file(file_path, content)

Created sample file: /Users/atrask/Desktop/SyftBoxBob/datasites/bob@openmined.org/company_data/financial_metrics.csv
Created sample file: /Users/atrask/Desktop/SyftBoxBob/datasites/bob@openmined.org/company_data/strategic_plan.txt


## Setting File Permissions for Alice

In [4]:
# Set permissions for Alice to access Bob's financial metrics
alice_permissions = bob_client.set_file_permissions(
    to_email=bob_client.client.email,  # Bob setting permissions on his own server
    user_email="alice@openmined.org",  # For Alice
    file_paths=[f"{bob_data_dir}/financial_metrics.csv"],  # Only allow access to this file
    operation="set"
)

print(f"Set Alice's permissions on Bob's server: {alice_permissions.success}")
print(f"Alice can access: {alice_permissions.allowed_files}")

Set Alice's permissions on Bob's server: True
Alice can access: ['/Users/atrask/Desktop/SyftBoxBob/datasites/bob@openmined.org/company_data/financial_metrics.csv']


## Testing Bob's Client with Own Data

In [5]:
prompt = "Analyze the financial metrics in the file. What's the growth trend? What quarter had the highest profit?"

response = bob_client.generate(
    to_email=bob_client.client.email,  # Bob querying his own data
    model='llama3',
    prompt=prompt,
    system="You are a financial analyst assistant. Provide concise insights based on the data.",
    temperature=0.7,
    files=[f"{bob_data_dir}/financial_metrics.csv"]  # Using Bob's file
)

print(f"\nResponse from Bob's LLM on his own data:\n{'-'*40}\n{response.response}\n{'-'*40}")


Response from Bob's LLM on his own data:
----------------------------------------
Error: User bob@openmined.org does not have permission to access file: /Users/atrask/Desktop/SyftBoxBob/datasites/bob@openmined.org/company_data/financial_metrics.csv
----------------------------------------


## Later: Adding More Permissions for Alice

In [6]:
# Add permission for Alice to see the strategic plan too
alice_more_permissions = bob_client.set_file_permissions(
    to_email=bob_client.client.email,
    user_email="alice@openmined.org",
    file_paths=[f"{bob_data_dir}/strategic_plan.txt"],  
    operation="add"  # Add to existing permissions instead of replacing
)

print(f"Updated Alice's permissions: {alice_more_permissions.success}")
print(f"Alice can now access: {alice_more_permissions.allowed_files}")

Updated Alice's permissions: True
Alice can now access: ['/Users/atrask/Desktop/SyftBoxBob/datasites/bob@openmined.org/company_data/strategic_plan.txt']


## Testing Connection to Alice's LLM (if available)

In [7]:
# This assumes Alice has already set up her client and granted permissions to Bob
try:
    # Check if Alice's datasite is available
    datasites = bob_client.list_datasites()
    alice_email = "alice@openmined.org"
    
    if alice_email in datasites:
        print(f"Alice's datasite is available. Bob can try to access Alice's data.")
        
        # List what files Bob has access to on Alice's datasite
        permitted_files = bob_client.list_permitted_files(
            to_email=alice_email,
            user_email=bob_client.client.email
        )
        
        if permitted_files and permitted_files.allowed_files:
            print(f"Bob has access to these files on Alice's datasite:")
            for file_path in permitted_files.allowed_files:
                print(f"  - {file_path}")
                
            # Try to query one of Alice's files Bob has permission to access
            sample_file = permitted_files.allowed_files[0]
            prompt = "Summarize the data in this file."
            
            response = bob_client.generate(
                to_email=alice_email,
                model='llama3',
                prompt=prompt,
                system="Provide a concise analysis of the data.",
                temperature=0.7,
                files=[sample_file]
            )
            
            print(f"\nResponse from Alice's LLM:\n{'-'*40}\n{response.response}\n{'-'*40}")
        else:
            print("Bob doesn't have permission to access any files on Alice's datasite yet.")
    else:
        print(f"Alice's datasite is not available.")
        print(f"Available datasites: {datasites}")
except Exception as e:
    print(f"Error when trying to access Alice's datasite: {e}")

Alice's datasite is available. Bob can try to access Alice's data.
Bob has access to these files on Alice's datasite:
  - /Users/atrask/Desktop/SyftBoxAlice/datasites/alice@openmined.org/private_data/patient_data.csv

Response from Alice's LLM:
----------------------------------------
The provided CSV file contains patient data from an imaginary healthcare system. The data includes:

* Patient ID (P001-P005)
* Age (range: 29-67 years old)
* Gender (F for female, M for male)
* Medical condition (Hypertension, Diabetes, Anxiety, Arthritis, Depression)
* Treatment type (Medication A-D, Therapy C-E)
* Outcome (Improved, Stable, No Change, Significantly Improved)

The data appears to be a small sample of patients with different medical conditions and treatment approaches. The outcomes suggest that some treatments were effective in improving patient symptoms, while others did not lead to significant changes.
----------------------------------------
